# Problem 212: Combined Volume of Cuboids

An <dfn>axis-aligned cuboid</dfn>, specified by parameters $\{(x_0, y_0, z_0), (dx, dy, dz)\}$, consists of all points $(X,Y,Z)$ such that $x_0 \le X \le x_0 + dx$, $y_0 \le Y \le y_0 + dy$ and $z_0 \le Z \le z_0 + dz$.  The volume of the cuboid is the product, $dx \times dy \times dz$.  The <dfn>combined volume</dfn> of a collection of cuboids is the volume of their union and will be less than the sum of the individual volumes if any cuboids overlap.

Let $C_1, \dots, C_{50000}$ be a collection of $50000$ axis-aligned cuboids such that $C_n$ has parameters

\begin{align}
x_0 &= S_{6n - 5} \bmod 10000 \nonumber \\
y_0 &= S_{6n - 4} \bmod 10000 \nonumber \\
z_0 &= S_{6n - 3} \bmod 10000 \nonumber \\
dx &= 1 + (S_{6n - 2} \bmod 399) \nonumber \\
dy &= 1 + (S_{6n - 1} \bmod 399) \nonumber \\
dz &= 1 + (S_{6n} \bmod 399) \nonumber
\end{align}

where $S_1,\dots,S_{300000}$ come from the "Lagged Fibonacci Generator":

For $1 \le k \le 55$, $S_k = [100003 - 200003k + 300007k^3] \pmod{1000000}$. \
For $56 \le k$, $S_k = [S_{k -24} + S_{k - 55}] \pmod{1000000}$.

Thus, $C_1$ has parameters $\{(7,53,183),(94,369,56)\}$, $C_2$ has parameters $\{(2383,3563,5079),(42,212,344)\}$, and so on.

The combined volume of the first $100$ cuboids, $C_1, \dots, C_{100}$, is $723581599$.

What is the combined volume of all $50000$ cuboids, $C_1, \dots, C_{50000}$?

In [1]:
import functools

@functools.cache
def S(k: int) -> int:
    if k <= 55:
        return (100003 - 200003*k + 300007*k*k*k) % 1000000
    else:
        return S(k-24) + S(k-55) % 1000000

def C(n: int):
    x0 = S(6*n-5) % 10000
    y0 = S(6*n-4) % 10000
    z0 = S(6*n-3) % 10000
    dx = 1 + (S(6*n-2) % 399)
    dy = 1 + (S(6*n-1) % 399)
    dz = 1 + (S(6*n) % 399)
    return [x0, y0, z0, dx, dy, dz]

S.cache_clear()
print("C(1) = ", C(1))
print("C(2) = ", C(2))

C(1) =  [7, 53, 183, 94, 369, 56]
C(2) =  [2383, 3563, 5079, 42, 212, 344]


In [59]:
from dataclasses import dataclass
from functools import cached_property
from typing import Tuple


@dataclass
class Cuboid:
    x0: int
    y0: int
    z0: int
    dx: int
    dy: int
    dz: int

    @cached_property
    def volume(self) -> int:
        return self.dx * self.dy * self.dz

    @property
    def xmin(self) -> int:
        return self.x0

    @property
    def ymin(self) -> int:
        return self.y0

    @property
    def zmin(self) -> int:
        return self.y0

    @cached_property
    def xmax(self) -> int:
        return self.x0 + self.dx

    @cached_property
    def ymax(self) -> int:
        return self.y0 + self.dy

    @cached_property
    def zmax(self) -> int:
        return self.z0 + self.dz

    def x_overlap(self, other: "Cuboid") -> Tuple[int, int, int]:
        xmin = max(self.xmin, other.xmin)
        xmax = min(self.xmax, other.xmax)
        dx = xmax - xmin
        return (xmin, xmax, dx)

    def y_overlap(self, other: "Cuboid") -> Tuple[int, int, int]:
        ymin = max(self.ymin, other.ymin)
        ymax = min(self.ymax, other.ymax)
        dy = ymax - ymin
        return (ymin, ymax, dy)

    def z_overlap(self, other: "Cuboid") -> Tuple[int, int, int]:
        zmin = max(self.zmin, other.zmin)
        zmax = min(self.zmax, other.zmax)
        dz = zmax - zmin
        return (zmin, zmax, dz)

    def is_overlap(self, other: "Cuboid") -> bool:
        if self.xmin >= other.xmax:
            return False
        elif self.xmax <= other.xmin:
            return False
        if self.ymin >= other.ymax:
            return False
        elif self.ymax <= other.ymin:
            return False
        if self.zmin >= other.zmax:
            return False
        elif self.zmax <= other.zmin:
            return False
        else:
            return True
        # _, _, d = self.x_overlap(other)
        # if d <= 0:
        #     return False
        # _, _, d = self.y_overlap(other)
        # if d <= 0:
        #     return False
        # _, _, d = self.z_overlap(other)
        # if d <= 0:
        #     return False
        # return True

    def overlap(self, other: "Cuboid") -> "Cuboid":
        xmin, _, dx = self.x_overlap(other)
        ymin, _, dy = self.y_overlap(other)
        zmin, _, dz = self.z_overlap(other)
        if dx <= 0 or dy <= 0 or dz <= 0:
            raise ValueError("No overlap")
        return Cuboid(xmin, ymin, zmin, dx, dy, dz)


c1 = Cuboid(0, 0, 0, 10, 10, 10)
c2 = Cuboid(10, 0, 0, 10, 10, 10)
c3 = Cuboid(4, 4, 4, 2, 2, 20)
print("c1.volume", c1.volume)
print("c1.overlaps(c2) =", c1.is_overlap(c2))
print("c1.overlap(c3) =", c1.overlap(c3))

c1.volume 1000
c1.overlaps(c2) = False
c1.overlap(c3) = Cuboid(x0=4, y0=4, z0=4, dx=2, dy=2, dz=6)


In [60]:
%timeit c1.is_overlap(c3)

300 ns ± 0.56 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [61]:
cuboids = [Cuboid(*C(n)) for n in range(1, 50000+1)]

In [63]:
sum([c.volume for c in cuboids])

400699965543

In [65]:
g1 = []
g1.append(cuboids.pop(0))

In [70]:
for _g1 in g1:
    index = []
    for i in range(len(cuboids)):
        if _g1.is_overlap(cuboids[i]):
            index.append(i)
    if len(index) == 0:
        break
    for i in index[::-1]:
        g1.append(cuboids.pop(i))

In [71]:
len(g1)

74

In [62]:
from itertools import combinations

for (i, (c1, c2)) in enumerate(combinations(cuboids, 2)):
    if i % 10**8 == 0:
        print(i, 50000*49999//2, i/50000/49999/2)
    if c1.is_overlap(c2):
        continue

0 1249975000 0.0
100000000 1249975000 0.02000040000800016
200000000 1249975000 0.04000080001600032
300000000 1249975000 0.06000120002400048
400000000 1249975000 0.08000160003200064
500000000 1249975000 0.1000020000400008


KeyboardInterrupt: 